In [1]:
import rebound
import reboundx
import numpy as np
import sys
import os
import re
import pandas as pd

df = pd.DataFrame(columns=('K','mag','filename'))

datapath = 'data/'
for filename in os.listdir(datapath):
    result = re.search(r"IC(.*)K(.*)mag(.*).bin", filename)
    if result:
        simID = int(result.group(1))
        K = float(result.group(2))
        mag = float(result.group(3))
        filename='IC{0}K{1:.4e}mag{2:.4e}.bin'.format(simID, K, mag)
        df.loc[simID] = [K,mag,filename]
        
df = df.sort_index()
df.tail()

,K,mag,filename
984,28.237,0.076254,IC984K2.8237e+01mag7.6254e-02.bin
986,201.970,0.052832,IC986K2.0197e+02mag5.2832e-02.bin
990,914.570,0.006452,IC990K9.1457e+02mag6.4517e-03.bin
995,53.041,0.903500,IC995K5.3041e+01mag9.0350e-01.bin
997,77.841,0.530970,IC997K7.7841e+01mag5.3097e-01.bin


In [2]:
from collections import OrderedDict
threebodyresonances = OrderedDict([(('b','c','d'),(2,3)),(('c','d','e'),(1,2)),(('d','e','f'),(2,3)),(('e','f','g'),(1,2)),(('f','g','h'),(1,1))])

def wrap(val):
    while val < 0:
        val += 2*np.pi
    while val > 2*np.pi:
        val -= 2*np.pi
    return val*180/np.pi

def libration_centers(row):
    sa = rebound.SimulationArchive(datapath+row['filename'])
    sim = sa[0]
    ps = sim.particles
    phi = {}
    for resonance in threebodyresonances.items():
        triad = resonance[0]
        p1 = triad[0]
        p2 = triad[1]
        p3 = triad[2]
        res = resonance[1]
        p = res[0]
        q = res[1]
        name = 'phi_'+p1+p2+p3
        phi[name] = wrap(p*ps[p1].l - (p+q)*ps[p2].l + q*ps[p3].l)
    return pd.Series(phi)

In [3]:
%%time
df = pd.concat([df, df.apply(libration_centers, axis=1)], axis=1)
df.tail()

/mnt/raid-cita/dtamayo/stability/lib/python3.4/site-packages/rebound/simulationarchive.py:63: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(BINARY_WARNINGS[1][0], RuntimeWarning)


CPU times: user 701 ms, sys: 1.97 s, total: 2.68 s
Wall time: 5.1 s


In [4]:
df.head()

,K,mag,filename,phi_bcd,phi_cde,phi_def,phi_efg,phi_fgh
0,125.210,0.139820,IC0K1.2521e+02mag1.3982e-01.bin,180.942875,178.786975,159.473438,72.594696,184.056444
2,74.471,0.001196,IC2K7.4471e+01mag1.1961e-03.bin,190.900879,300.483438,153.734757,76.238337,183.543375
4,859.130,0.043822,IC4K8.5913e+02mag4.3822e-02.bin,205.119741,304.742293,140.799400,79.610096,179.641168
6,610.550,0.009907,IC6K6.1055e+02mag9.9069e-03.bin,348.877103,5.394857,297.858031,224.000963,143.139271
7,14.211,0.218650,IC7K1.4211e+01mag2.1865e-01.bin,243.790429,71.575234,120.974760,77.358878,181.352829


In [5]:
df.to_csv("csvs/librationcenters.csv", encoding='ascii')